In [43]:
class Value():
    def __init__(self, data, _children = (), _op = "" ):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children) # -> this is implemented like this for efficiency ? -> how ?
        self.op = _op
        
    def __repr__(self):        
        return f"Value(data = {self.data})"
        
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), "+")
        
        def _backward(): # closure
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        self._backward = _backward
        
        return out

    def __neg__(self):
        return self*-1
    
    def __sub__(self, other):
        return self.data + (-other.data)
        
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        self._backward = _backward
            
        return out

    def __rmul__(self, other):
         return self * other 


    def __pow__(self, other):
        
        out = Value(self.data**other, (self,), '^')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.data
        t = ((math.pow(2*x)-1)) / (math.pow(2*x)-1)

        def _backward():
            self.grad += (1- t**2) * out.grad
        self._backward = _backward
        out = Value(r.data, (self, ), "tanh")

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [44]:
a = Value(2.0)
b = Value(3.0)
b-a

TypeError: unsupported operand type(s) for -: 'Value' and 'Value'